In [1]:
from tkinter import *
import numpy as np  


Alpha=['A','B','C','D','E','F','G','H']
Beta=['1','2','3','4','5','6','7','8']

'''figure_classes = {
    'pawn': pawn,
    'rook': rook,
    'knight': knight,
    'bishop': bishop,
    'king': king,
    'queen': queen,
    '0': zero,
    0: zero
}'''
coder = {
    'pawn_1':   '\u2659',    'pawn_2':   '\u265F',
    'rook_1':   '\u2656',    'rook_2':   '\u265C',
    'knight_1': '\u2658',    'knight_2': '\u265E',
    'bishop_1': '\u2657',    'bishop_2': '\u265D',
    'king_1':   '\u2654',    'king_2':   '\u265A',
    'queen_1':  '\u2655',    'queen_2':  '\u265B',
    '0_0':      ''
}
start_positions = tuple([
    tuple(['rook_2', 'knight_2', 'bishop_2',  'queen_2','king_2', 'bishop_2', 'knight_2', 'rook_2']),
    tuple(['pawn_2' for i in range(8)]),
    tuple(['0_0' for i in range(8)]),
    tuple(['0_0' for i in range(8)]),
    tuple(['0_0' for i in range(8)]),
    tuple(['0_0' for i in range(8)]),
    tuple(['pawn_1' for i in range(8)]),
    tuple(['rook_1', 'knight_1', 'bishop_1',  'queen_1', 'king_1', 'bishop_1', 'knight_1', 'rook_1']),
])
get_code = np.vectorize(coder.get)

selected_buttons = []
changed_for_move = []

def button_click(button, position):
    """Обработчик нажатия кнопки."""
    global selected_buttons

    # Сбрасываем цвет предыдущей кнопки
    if len(selected_buttons):
        if selected_buttons[-1]:
            selected_buttons[-1].configure(bg=selected_buttons[-1].default_color)
            
    if len(changed_for_move):
        for i in changed_for_move:
            i.configure(i.default_color)
    
    # Устанавливаем цвет текущей кнопки
    button.configure(bg="yellow")
    
    selected_buttons.append(button)
    
    #for i in predict_list(position)

    
    
    
    print(f"{position}")

def create_chess_board(canvas,cell_size,rows=8, cols=8,plus=2,font=None):
    
    if font==None:
        font = ('Times New Roman', int((cell_size)**0.8))

    for row in range(rows):
        for col in range(cols):
            # Определяем цвет клетки
            color = "white" if (row + col) % 2 == 0 else "black"
            
            position = f"{Alpha[col]}{Beta[row]}"
            
            
            # Создаем кнопку
            btn = Button(
                canvas, 
                text = get_code(start_positions)[row][col],
                font=font,
                fg="black" if color == "white" else "white",
                bg=color, 
                activebackground=color,
                 # Захват кнопки и координат
            )
            btn.configure(command=lambda b=btn, pos=position: button_click(b, pos))
            # Сохраняем исходный цвет кнопки
            btn.default_color = color
            
            # Размещаем кнопку на Canvas
            canvas.create_window((col+plus) * cell_size, (row+plus) * cell_size, 
                                 window=btn, width=cell_size, height=cell_size)
            
    # Добавляем буквенные обозначения (сверху и снизу)
    for col in range(cols):
        letter = Alpha[col]
        canvas.create_text(
            (col + plus) * cell_size, 1 * cell_size, 
            text=letter, font=font
        )
        canvas.create_text(
            (col + plus) * cell_size, (rows + plus) * cell_size, 
            text=letter, font=font
        )
    
    # Добавляем числовые обозначения (слева и справа)
    for row in range(rows):
        number = Beta[row]
        canvas.create_text(
            1 * cell_size, (row + plus) * cell_size, 
            text=number, font=font
        )
        canvas.create_text(
            (cols + plus) * cell_size, (row + plus) * cell_size, 
            text=number, font=font
        )
    

    return 

# Создаем главное окно
root = Tk()
root.title("Шахматная доска из кнопок")

cell_size=40
rows = 8
plus = 2
# Задаем размеры Canvas
canvas_size = (cell_size)*(rows+1+plus)  # Шахматная доска 8x8 с размерами клетки 50x50 пикселей
canvas = Canvas(root, width=canvas_size, height=canvas_size)
canvas.pack()

# Создаем шахматную доску
create_chess_board(canvas,cell_size,plus=plus)

# Запускаем главное окно
root.mainloop()

G7
F8
F7
G8
H8
H7


In [ ]:
import collections

class Field():
    def __init__(self, logic=None):
        """
        Initialize a new game field instance.

        Parameters
        ----------
        logic : list of lists, optional
            A two-dimensional list representing the game board. The default is None.

        Attributes
        ----------
        logic : list of lists
            A two-dimensional list representing the game board.
        """
        self.logic = logic if logic else self.first_generation()

    def first_generation(self):
        """
        Method for generating the initial state of the board.

        Generates a two-dimensional list representing the game board, based on the start_positions tuple.

        Returns
        -------
        logic : list of lists
            A two-dimensional list representing the initial state of the game board.
        """
        logic = [[0 for i in range(8)] for j in range(8)] # <=== Логическое представление игрового поля, его модель

        for i in range(8):
            for j in range(8):
                logic[i][j] = figure_classes[start_positions[i][j][:-2]](figure=start_positions[i][j][:-2],figure_color=int(start_positions[i][j][-1]), X=i, Y=j)

        return logic
    
    def predict_danger(self, logic, play, l=0):
        """
        Predicts the positions that can be attacked by the opponent in the given turn.

        Parameters
        ----------
        logic : list of lists
            A two-dimensional list representing the game board.
        play : int
            The index of the player currently making the move (0 or 1).
        l : int
            The number of the move being made.

        Returns
        -------
        list
            A list of coordinates that can be attacked by the opponent in the given turn.
        """
        eat_predict_list = []
        for row in range(8):
            for x in range(8):
                if logic[row][x].figure_color!=play:
                    predictions = logic[row][x].predict([row,x,logic,play+1],1)
                    for i in predictions:
                        if i[0] in [_ for _ in range(8)] and i[1] in [_ for _ in range(8)]:
                            if logic[i[0]][i[1]].figure_color==play:
                                eat_predict_list.append(i)
                    
        return eat_predict_list

                
                
    def render(self, moves, eaten,predictlist=[], eat_predictlist=[],danger_predictlist=[],chekking_list=[],linking_dict=dict()):
        """
        Renders the current state of the game board.

        Parameters
        ----------
        moves : int
            The number of the current move.
        eaten : list of str
            A list of eaten figures in the format 'figure_color', where 'color' is '1' for white and '2' for black.
        predictlist : list of tuples, optional
            A list of coordinates of possible moves without eating.
        eat_predictlist : list of tuples, optional
            A list of coordinates of possible moves with eating.
        danger_predictlist : list of tuples, optional
            A list of coordinates of figures that can be attacked.
        chekking_list : list of tuples, optional
            A list of coordinates of figures that can attack the king.
        linking_dict : dict, optional
            A dictionary of coordinates of figures that can attack the king.

        Returns
        -------
        None
        """
        
        color=(moves)%2+1
        eaten_w=[]
        eaten_b=[]
        for i in range(len(eaten)):
            if eaten[i][-1]=='1':
                eaten_w.append(coder[eaten[i]])
            else:
                eaten_b.append(coder[eaten[i]])

        co_eaten_w=dict(collections.Counter(eaten_w))
        co_eaten_b=dict(collections.Counter(eaten_b))
        if len(predictlist)!=0 or len(eat_predictlist)!=0:
            if color == 1:

                #### Выведем буквы

                print(' '.center(1),end='',)
                for i in range(len(Alpha)):
                    print(('\033[33m'+Alpha[i]+'\033[0m').center(11), end='')
                
                #### Выведем метку для съеденных фигур
                    
                print('\033[33mEaten figures:\033[0m'.center(60))

                #### начнем выводить фигуры

                for row in range(len(self.logic)):
                    for x in range(len(self.logic[row])):
                        if x==0: #### Выведем Цифры
                            print(('\033[33m'+Beta[7-row]+'\033[0m'+' ').center(2), end='')
                        if tuple([row,x]) in predictlist: #### Возможные перемещения без съедания
                            print((f'\033[42m'+str(self.logic[row][x])+'\033[0m').center(11),end='')

                        elif tuple([row,x]) in eat_predictlist:  #### Возможные перемещения со съеданиями
                            print((f'\033[43m'+str(self.logic[row][x])+'\033[0m').center(11),end='')

                        elif tuple([row,x]) in danger_predictlist: #### Фигуры, которые могут быть атакованы
                            print((f'\033[41m'+str(self.logic[row][x])+'\033[0m').center(11),end='')

                        elif tuple([row,x]) in chekking_list: #### Выдeеляет клетки, которые могут атаковать короля
                            print((f'\033[45m'+str(self.logic[row][x])+'\033[0m').center(11),end='')
                        else:  #### Обычнй вывод фигур 
                            print((f'\033[4{7*((1+row+x)%2)}m'+str(self.logic[row][x])+'\033[0m').center(11),end='')

                        if x==len(self.logic[row])-1:
                            print((' \033[33m'+Beta[7-row]+'\033[0m'+' ').center(2), end='')
                            if row == 0:
                                print('\033[1m\033[40m\033[37mWhite\033[0m'.center(40),end='')
                                print('\033[1m\033[47m\033[30mBlack\033[0m'.center(36))
                            else:
                                try:
                                    print((list(co_eaten_w.keys())[row-1] +'x'+str(co_eaten_w[list(co_eaten_w.keys())[row-1]])).center(20),end='')
                                except KeyError:
                                    print(''.center(20),end='')
                                except IndexError:
                                    print(''.center(20),end='')
                                finally:
                                    try:
                                        print((list(co_eaten_b.keys())[row-1] +'x'+str(co_eaten_b[list(co_eaten_b.keys())[row-1]])).center(21),end='')
                                    except KeyError:
                                        print(' ',end='')
                                    except IndexError:
                                        print(' ',end='')
                                    finally:
                                        print()
                print(' '.center(1),end='')
                for i in range(len(Alpha)):
                    print(('\033[33m'+Alpha[i]+'\033[0m').center(11), end='')
                print(f'Moves: {moves}'.center(19),end='')
                print("\033[1m\033[40m\033[37mWhite's move\033[0m".center(20))
            else:
                print(' '.center(1),end='')
                for i in range(len(Alpha)):
                    print(('\033[33m'+Alpha[7- i]+'\033[0m').center(11), end='')
                print('\033[33mEaten figures:\033[0m'.center(60), end='')
                print()
                for row in range(len(self.logic)):
                    for x in range(len(self.logic[row])):
                        if x==0:
                            print(('\033[33m'+Beta[row]+'\033[0m'+' ').center(2), end='')
                        if tuple([7-row,7-x]) in predictlist:
                            print((f'\033[42m'+str(self.logic[7-row][7-x])+'\033[0m').center(11),end='')
                        elif tuple([7-row,7-x]) in eat_predictlist:
                            print((f'\033[43m'+str(self.logic[7-row][7-x])+'\033[0m').center(11),end='')
                        elif tuple([7-row,7-x]) in danger_predictlist:
                            print((f'\033[41m'+str(self.logic[7-row][7-x])+'\033[0m').center(11),end='')
                        elif tuple([7-row,7-x]) in chekking_list:
                            print((f'\033[45m'+str(self.logic[7-row][7-x])+'\033[0m').center(11),end='')
                        else:
                            print((f'\033[4{7*((1+row+x)%2)}m'+str(self.logic[7-row][7-x])+'\033[0m').center(11),end='')
                        if x==len(self.logic[row])-1:
                            print((' \033[33m'+Beta[row]+'\033[0m'+' ').center(2),end='')
                            if row == 0:
                                print('\033[1m\033[40m\033[37mWhite\033[0m'.center(40),end='')
                                print('\033[1m\033[47m\033[30mBlack\033[0m'.center(36))
                            else:
                                try:
                                    print((list(co_eaten_w.keys())[row-1] +'x'+str(co_eaten_w[list(co_eaten_w.keys())[row-1]])).center(20),end='')
                                except KeyError:
                                    print(''.center(20),end='')
                                except IndexError:
                                    print(''.center(20),end='')
                                finally:
                                    try:
                                        print((list(co_eaten_b.keys())[row-1] +'x'+str(co_eaten_b[list(co_eaten_b.keys())[row-1]])).center(21),end='')
                                    except KeyError:
                                        print(' ',end='')
                                    except IndexError:
                                        print(' ',end='')
                                    finally:
                                        print()
                print(' '.center(1),end='')
                for i in range(len(Alpha)):
                    print(('\033[33m'+Alpha[7-i]+'\033[0m').center(11), end='')
                print(f'Moves: {moves}'.center(19),end='')
                print("\033[1m\033[47m\033[30mBlack's move\033[0m".center(20))
